In [4]:
!pip3 install datasets
!pip3 install sentencepiece
!pip3 install seqeval
!pip install transformers[torch]
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 562.2 kB/s eta 0:00:00 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=60d5bae038653801976f94497e920635f10a1a79a6524ea29e4459ef192984d0
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [5]:
import datasets

In [7]:
# Let's download the Naampadam (Indic NER) dataset
from datasets import ClassLabel, load_dataset, load_metric, DownloadMode

lang='hi'

train_dataset = load_dataset('ai4bharat/naamapadam', lang , split = 'train[:200000]')
validation_dataset = load_dataset('ai4bharat/naamapadam', lang , split = 'validation')
test_dataset = load_dataset('ai4bharat/naamapadam', lang , split = 'test')



In [8]:
features = train_dataset.features
print(features)

{'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None), 'ner_tags': Sequence(feature=ClassLabel(num_classes=7, names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC'], id=None), length=-1, id=None)}


In [9]:
text_column_name = "tokens"
label_column_name = "ner_tags"

label_list = features[label_column_name].feature.names

label_to_id = {label_list[i]: features[label_column_name].feature.str2int( label_list[i] ) for i in range(len(label_list))}

print(label_to_id)

num_labels = len(label_list)


{'O': 0, 'B-PER': 1, 'I-PER': 2, 'B-ORG': 3, 'I-ORG': 4, 'B-LOC': 5, 'I-LOC': 6}


In [10]:
from transformers import AutoModelForTokenClassification, AutoConfig, AutoTokenizer, TrainingArguments, Trainer, DataCollatorForTokenClassification, EarlyStoppingCallback, IntervalStrategy
import numpy as np

config = AutoConfig.from_pretrained('ai4bharat/indic-bert', num_labels=num_labels, finetuning_task='ner')
tokenizer = AutoTokenizer.from_pretrained("ai4bharat/indic-bert")
model = AutoModelForTokenClassification.from_pretrained('ai4bharat/indic-bert', num_labels=num_labels )

2024-03-11 04:59:22.506670: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-11 04:59:22.506768: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-11 04:59:22.671181: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


config.json:   0%|          | 0.00/507 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/5.65M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/135M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of AlbertForTokenClassification were not initialized from the model checkpoint at ai4bharat/indic-bert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
# Tokenize all texts and align the labels with them.
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples[text_column_name],
        padding="max_length",
        truncation=True,
        max_length=256,
        # We use this argument because the texts in our dataset are lists of words (with a label for each word).
        is_split_into_words=True,
    )
    # print(tokenized_inputs)
    labels = []
    for i, label in enumerate(examples[label_column_name]):
        # print('=====')
        # print('{} {}'.format(i,label)) #ak
        word_ids = tokenized_inputs.word_ids(batch_index=i)

        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx

        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    # print(tokenized_inputs)
    return tokenized_inputs

In [12]:
tokenized_train_dataset = train_dataset.map(
    tokenize_and_align_labels,
    batched=True,
)
print(tokenized_train_dataset)

  0%|          | 0/200 [00:00<?, ?ba/s]

Dataset({
    features: ['tokens', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 200000
})


In [13]:
# Problem is fixed now!!
print(len(tokenized_train_dataset[0]['ner_tags']))
print(len(tokenized_train_dataset[0]['labels']))
print(len(tokenized_train_dataset[0]['input_ids']))

30
256
256


In [14]:
# Test Dataset
tokenized_test_dataset = test_dataset.map(
    tokenize_and_align_labels,
    batched=True,
)
print(tokenized_test_dataset)

  0%|          | 0/1 [00:00<?, ?ba/s]

Dataset({
    features: ['tokens', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 867
})


In [15]:
# Validation DataSet
tokenized_validation_dataset = validation_dataset.map(
    tokenize_and_align_labels,
    batched=True,
)
print(tokenized_validation_dataset)

  0%|          | 0/14 [00:00<?, ?ba/s]

Dataset({
    features: ['tokens', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 13460
})


In [16]:
# Data Collator
data_collator = DataCollatorForTokenClassification(tokenizer)

In [17]:
# Metric to see how my model is performing For NER we use seqeval
metric = datasets.load_metric("seqeval")

In [18]:
# Seeing the working of seqeval
example_text = train_dataset[0]
label_list = train_dataset.features["ner_tags"].feature.names
label_list

labels = [label_list[i] for i in example_text["ner_tags"]]
print(labels)

metric.compute(predictions=[labels] , references=[labels])

['O', 'O', 'O', 'O', 'B-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


{'PER': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'overall_precision': 1.0,
 'overall_recall': 1.0,
 'overall_f1': 1.0,
 'overall_accuracy': 1.0}

In [23]:
# Metrics
metric = load_metric("seqeval")

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    # Unpack nested dictionaries
    final_results = {}
    for key, value in results.items():
        if isinstance(value, dict):
            for n, v in value.items():
                final_results[f"{key}_{n}"] = v
        else:
            final_results[key] = value
    return final_results

In [24]:
# Define Training Args
args = TrainingArguments(
    output_dir=f"model/upos",
    overwrite_output_dir=True,
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=10,
    per_device_eval_batch_size=10,
    num_train_epochs=5,
    weight_decay=0.01,
    save_total_limit=1,
)

In [25]:
trainer = Trainer(
   model,
   args,
   train_dataset=tokenized_train_dataset,
   eval_dataset=tokenized_validation_dataset,
   data_collator=data_collator,
   tokenizer=tokenizer,
   compute_metrics=compute_metrics,
)

In [26]:
train_result = trainer.train()
metrics = train_result.metrics

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Loc Precision,Loc Recall,Loc F1,Loc Number,Org Precision,Org Recall,Org F1,Org Number,Per Precision,Per Recall,Per F1,Per Number,Overall Precision,Overall Recall,Overall F1,Overall Accuracy
1,0.214300,0.222826,0.741471,0.795848,0.767698,10213,0.623049,0.587268,0.604629,9786,0.714741,0.784538,0.748015,10568,0.697351,0.725161,0.710984,0.930108
2,0.197200,0.205257,0.783299,0.791736,0.787495,10213,0.625890,0.628449,0.627167,9786,0.777367,0.775454,0.776409,10568,0.730866,0.733831,0.732345,0.935464
3,0.178200,0.201126,0.777536,0.804563,0.790819,10213,0.661362,0.624259,0.642275,9786,0.771489,0.790689,0.780971,10568,0.740371,0.742042,0.741205,0.936835
4,0.160700,0.201779,0.767551,0.815725,0.790905,10213,0.669483,0.615369,0.641286,9786,0.767474,0.791730,0.779413,10568,0.738838,0.743285,0.741055,0.937410
5,0.144300,0.205040,0.777424,0.813277,0.794947,10213,0.651246,0.632945,0.641965,9786,0.772909,0.787755,0.780261,10568,0.737099,0.746720,0.741878,0.937541


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead u

In [38]:
sumoff1score=metrics['test_LOC_f1']+metrics['test_ORG_f1']+metrics['test_PER_f1']
macrof1=sumoff1score/3
print(macrof1)

0.7629731655133588


In [37]:
print(metrics)

{'test_loss': 0.16180792450904846, 'test_LOC_precision': 0.7792421746293245, 'test_LOC_recall': 0.7703583061889251, 'test_LOC_f1': 0.7747747747747749, 'test_LOC_number': 614, 'test_ORG_precision': 0.6590909090909091, 'test_ORG_recall': 0.7180952380952381, 'test_ORG_f1': 0.6873290793072013, 'test_ORG_number': 525, 'test_PER_precision': 0.8112058465286236, 'test_PER_recall': 0.8430379746835444, 'test_PER_f1': 0.8268156424581005, 'test_PER_number': 790, 'test_overall_precision': 0.758, 'test_overall_recall': 0.7858994297563504, 'test_overall_f1': 0.7716976329854925, 'test_overall_accuracy': 0.9495802543608305, 'test_runtime': 7.8217, 'test_samples_per_second': 110.845, 'test_steps_per_second': 5.625}


In [44]:
!ls

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


directory.zip  model_indic_bert_5Epoch_2lakh  wandb
model	       model_indic_bert_5Epoch_lakh


In [43]:
# Save Model
trainer.save_model("model_indic_bert_5Epoch_2lakh")

In [45]:
import zipfile
import os
from IPython.display import FileLink

def zip_dir(directory = os.curdir, file_name = 'directory.zip'):
    """
    zip all the files in a directory
    
    Parameters
    ___
    directory: str
        directory needs to be zipped, defualt is current working directory
        
    file_name: str
        the name of the zipped file (including .zip), default is 'directory.zip'
        
    Returns
    ___
    Creates a hyperlink, which can be used to download the zip file)
    """
    os.chdir(directory)
    zip_ref = zipfile.ZipFile(file_name, mode='w')
    for folder, _, files in os.walk(directory):
        for file in files:
            if file_name in file:
                pass
            else:
                zip_ref.write(os.path.join(folder, file))

    return FileLink(file_name)

In [46]:
zip_dir()

/kaggle/working/directory.zip

In [42]:
metrics=trainer.evaluate()
print(metrics)

{'eval_loss': 0.20503978431224823, 'eval_LOC_precision': 0.7774241856982403, 'eval_LOC_recall': 0.8132771957309312, 'eval_LOC_f1': 0.7949466430588122, 'eval_LOC_number': 10213, 'eval_ORG_precision': 0.6512459257701608, 'eval_ORG_recall': 0.6329450235029634, 'eval_ORG_f1': 0.6419650722910297, 'eval_ORG_number': 9786, 'eval_PER_precision': 0.7729087364218735, 'eval_PER_recall': 0.7877554882664648, 'eval_PER_f1': 0.7802614930409111, 'eval_PER_number': 10568, 'eval_overall_precision': 0.7370987534715494, 'eval_overall_recall': 0.74672031929859, 'eval_overall_f1': 0.7418783417028262, 'eval_overall_accuracy': 0.937541491116793, 'eval_runtime': 121.1628, 'eval_samples_per_second': 111.09, 'eval_steps_per_second': 5.555, 'epoch': 5.0}


In [48]:
# Evaluation of Model
predictions, labels, metrics = trainer.predict(tokenized_test_dataset)
print(metrics)
specific_results = {}
for key in metrics:
    if 'test_precision' in key:
      specific_results['test_precision'] = metrics[key]
    elif 'test_recall' in key:
      specific_results['test_recall'] = metrics[key]
    elif 'test_f1' in key:
      specific_results['test_f1'] = metrics[key]
print(specific_results)

{'test_loss': 0.16180792450904846, 'test_LOC_precision': 0.7792421746293245, 'test_LOC_recall': 0.7703583061889251, 'test_LOC_f1': 0.7747747747747749, 'test_LOC_number': 614, 'test_ORG_precision': 0.6590909090909091, 'test_ORG_recall': 0.7180952380952381, 'test_ORG_f1': 0.6873290793072013, 'test_ORG_number': 525, 'test_PER_precision': 0.8112058465286236, 'test_PER_recall': 0.8430379746835444, 'test_PER_f1': 0.8268156424581005, 'test_PER_number': 790, 'test_overall_precision': 0.758, 'test_overall_recall': 0.7858994297563504, 'test_overall_f1': 0.7716976329854925, 'test_overall_accuracy': 0.9495802543608305, 'test_runtime': 7.7884, 'test_samples_per_second': 111.32, 'test_steps_per_second': 5.649}
{}


In [50]:
for key,value in specific_results.items():
    print(key ,value)
print(metrics)
# sum_of_f1_scores=metrics['test_LOC_f1']+metrics['test_ORG_f1']+metrics['test_PER_f1']
# macro_f1=sum_of_f1_scores/3
print('\n Macro f1 score::',macro_f1)

{'test_loss': 0.16180792450904846, 'test_LOC_precision': 0.7792421746293245, 'test_LOC_recall': 0.7703583061889251, 'test_LOC_f1': 0.7747747747747749, 'test_LOC_number': 614, 'test_ORG_precision': 0.6590909090909091, 'test_ORG_recall': 0.7180952380952381, 'test_ORG_f1': 0.6873290793072013, 'test_ORG_number': 525, 'test_PER_precision': 0.8112058465286236, 'test_PER_recall': 0.8430379746835444, 'test_PER_f1': 0.8268156424581005, 'test_PER_number': 790, 'test_overall_precision': 0.758, 'test_overall_recall': 0.7858994297563504, 'test_overall_f1': 0.7716976329854925, 'test_overall_accuracy': 0.9495802543608305, 'test_runtime': 7.7884, 'test_samples_per_second': 111.32, 'test_steps_per_second': 5.649}

 Macro f1 score:: 0.7629731655133588
